In [1]:
import torch
#https://www.kaggle.com/datasets/cristiangarcia/pointcloudmnist2d?select=train.csv
'''
Point Cloud Mnist 2D,
'''

In [37]:
D_in, H, D_out = 1053, 150, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [3]:
outputs = model(torch.randn(64, 1053))
outputs.shape

torch.Size([64, 10])

In [48]:
'''
Point Cloud Mnist 2D,
'''
#https://www.kaggle.com/datasets/cristiangarcia/pointcloudmnist2d?select=train.csv
import numpy as np
data= np.loadtxt('/content/train.csv', delimiter=',',skiprows=1)
# Divide las columnas en X (características) y Y (etiquetas)
X, Y =data[:,1:],data[:,0]
print(X.shape)
print(Y.shape)



(60000, 1053)
(60000,)


In [7]:
print(Y)
print(X)

[5. 0. 4. ... 5. 6. 8.]
[[ 17.  10. 249. ...  -1.  -1.  -1.]
 [ 22.  17.  50. ...  -1.  -1.  -1.]
 [  8.  12. 207. ...  -1.  -1.  -1.]
 ...
 [ 14.  13. 253. ...  -1.  -1.  -1.]
 [ 10.  14. 253. ...  -1.  -1.  -1.]
 [  7.   6. 254. ...  -1.  -1.  -1.]]


In [8]:
# normalización y split

#X_train = X[:7000] / 255.,
#X_test=X[7000:] / 255.,
#y_train= Y[:7000].astype(np.int)
#y_test = Y[7000:].astype(np.int)
X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)

<ipython-input-8-9771e080d068>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)


In [ ]:
#model.to("cuda")

In [9]:
# función de pérdida y derivada
def softmax(x):#usada en la conversion de puntajes y el uso de probabilidades
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):#calcula la perdida de la entropia
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))#calcular la perdida
    loss = loss.mean()#calcular la media de la perdias
    return loss

In [11]:
print(X)
print(X_train)

[[ 17.  10. 249. ...  -1.  -1.  -1.]
 [ 22.  17.  50. ...  -1.  -1.  -1.]
 [  8.  12. 207. ...  -1.  -1.  -1.]
 ...
 [ 14.  13. 253. ...  -1.  -1.  -1.]
 [ 10.  14. 253. ...  -1.  -1.  -1.]
 [  7.   6. 254. ...  -1.  -1.  -1.]]
[[ 0.06666667  0.03921569  0.97647059 ... -0.00392157 -0.00392157
  -0.00392157]
 [ 0.08627451  0.06666667  0.19607843 ... -0.00392157 -0.00392157
  -0.00392157]
 [ 0.03137255  0.04705882  0.81176471 ... -0.00392157 -0.00392157
  -0.00392157]
 ...
 [ 0.05882353  0.07843137  0.66666667 ... -0.00392157 -0.00392157
  -0.00392157]
 [ 0.0627451   0.01960784  0.99215686 ... -0.00392157 -0.00392157
  -0.00392157]
 [ 0.04313725  0.05490196  0.77254902 ... -0.00392157 -0.00392157
  -0.00392157]]


In [50]:
# convertimos datos a tensores y copiamos en gpu

#X_t = torch.from_numpy(X_train).float()
#Y_t = torch.from_numpy(y_train).long()
#print(X_t.shape)
#print(Y_t.shape)

torch.Size([7000, 784])
torch.Size([7000])


In [43]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float()#los datos de entrenamiento se convierten en tensores
Y_t = torch.from_numpy(y_train).long()#los datos de entrenamiento se convierten en tensores

# bucle entrenamiento
epochs = 150
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/150 Loss 1.98264
Epoch 20/150 Loss 1.97514
Epoch 30/150 Loss 1.96813
Epoch 40/150 Loss 1.96403
Epoch 50/150 Loss 1.96286
Epoch 60/150 Loss 1.96047
Epoch 70/150 Loss 1.95868
Epoch 80/150 Loss 1.95775
Epoch 90/150 Loss 1.95643
Epoch 100/150 Loss 1.95547
Epoch 110/150 Loss 1.95474
Epoch 120/150 Loss 1.95392
Epoch 130/150 Loss 1.95312
Epoch 140/150 Loss 1.95227
Epoch 150/150 Loss 1.95163


In [44]:
from sklearn.metrics import accuracy_score #libreria para calcular la precisión de las predicciones

def evaluate(x):
    model.eval()#poner el modelo en modo evaluacion
    y_pred = model(x) # realiza una inferencia en el modelo usando el tensor de entrada X para obtenera las predicciones
    y_probas = softmax(y_pred)#para obtener la probabilidades de clase, Convierte las puntuaciones del modelo en distribuciones de porbabilidad
    return torch.argmax(y_probas, axis=1)#encontrar la clase con la probabilidad mas alta para cada modelo

y_pred = evaluate(torch.from_numpy(X_test).float())#Realiza la evaluación del modelo en el conjunto de datos de prueba
accuracy_score(y_test, y_pred.cpu().numpy())

0.2622

In [45]:
from sklearn.metrics import accuracy_score

# ... (entrenamiento del modelo y carga de datos)

# Evaluar el modelo en el conjunto de datos de prueba
y_pred = evaluate(torch.from_numpy(X_test).float())

# Convertir las predicciones en etiquetas binarias (0 o 1)
y_pred_binary = (y_pred > 0.5).long()

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred_binary.cpu().numpy())
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 15.91%


Optimizador y funcion de perdida

-criterion = torch.nn.CrossEntropyLoss()

optimizador del descenso por el gradiente

-optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

In [47]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

criterion = torch.nn.CrossEntropyLoss()#funcion perdida
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)#optimizador

epochs = 150
log_each = 10
l = []
model.train()#poner el modelo en entrenamiento
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()#antes sobre el modelo ahora sobre el optimizador

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/150 Loss 2.21813
Epoch 20/150 Loss 2.16521
Epoch 30/150 Loss 2.12431
Epoch 40/150 Loss 2.09428
Epoch 50/150 Loss 2.07114
Epoch 60/150 Loss 2.05433
Epoch 70/150 Loss 2.04175
Epoch 80/150 Loss 2.03191
Epoch 90/150 Loss 2.02399
Epoch 100/150 Loss 2.01744
Epoch 110/150 Loss 2.01208
Epoch 120/150 Loss 2.00751
Epoch 130/150 Loss 2.00356
Epoch 140/150 Loss 2.00009
Epoch 150/150 Loss 1.99702


0.2654